# DS 3000 HW 5 

Due: Sunday July 21st @ 11:59 PM EST

### Submission Instructions
Submit this `ipynb` file and the a `PDF` file included with the coding results to Gradescope (this can also be done via the assignment on Canvas).  To ensure that your submitted files represent your latest code, make sure to give a fresh `Kernel > Restart & Run All` just before uploading the files to gradescope. 

**Notice that this is a group assignment. Each group only need to submit one copy and when you submit the work, please include everyone in your group.**

### Tips for success
- Start early
- Make use of Piazza
- Make use of Office hour
- Remember to use cells and headings to make the notebook easy to read (if a grader cannot find the answer to a problem, you will receive no points for it)
- Under no circumstances may one student view or share their ungraded homework or quiz with another student [(see also)](http://www.northeastern.edu/osccr/academic-integrity), though you are welcome to **talk about** (not show each other) the problems.

## Project proposal

For this course, we aim to complete a data analysis project about the the game [Palworld](https://en.wikipedia.org/wiki/Palworld). To help you start with the project, here are a couple of things you need to consider and work on to get a clean data for later analysis. 

To start with the project, please take some time to get familiar with the game. You don't need to play it but please at least know the basic terminologies, like what is a Pal. (And also, if you do play it, please do not spend too much time on it.)

The two recommended database is [https://palworld.gg/](https://palworld.gg/) and [https://paldb.cc/en/](https://paldb.cc/en/). You can use either, or both, or some other database about the Palworld. 

### Part 1.1 (10 points)

Please list 2-3 questions you may be interested to study with the Palworld database. It can be anything related in the game, like the Pals, items or constructions. Some potential question structures can be: 
- Are `A` and `B` related? How they are related?
- Which features may affect `C`'s change?
- If I need a higher `D`, which features may have a lower/higher value?
- Based on `E` and `F`, which items/pals are similar?
- I need to predict the value for `G`, which features I need to consider?

1. What factors influence the rarity of a PAL?
2. Which types of work are the most common?
3. Which element has the rarest PAL's?

### Part 1.2 (20 points)

Based on the questions we proposed in the part 1.1, what features we may need to include in the analysis? Check the websites, which website has those information? **You need to pick at least 8 features for analysis.** We recommend a mix of numerical (numbers etc.) and categorical (level etc.) features. Is there any other features that you think it may be important but hard to extract or find on the website (can be something in or not in the game)?

1. Rarity
2. Element type
3. Work type
4. Abilities
5. Stats
6. Size
7. names
8. Index (numerical identifier) 

### Part 1.3 (20 points)

Suppose you do have all the features you mentioned in part 1.2. List 3-4 data visulizations you can make with those features. You do not need to make those visulizations here. Just describe the type of the visualizations (histogram, scatter plot etc. ), which features are involved, will there any hover data or color being added, and **discuss how these data visualizations may be related (or even answer) to your questions in part 1.1**. 

1. Histogram representing work type frequencies. We would use the work type feature
2. Scatterplots for each stat, which graphs the stat against the rarity of each PAL. We would use this to see which stat has the highest coorelation to rarity. This visualization would use the stats, rarity, names, and index features.
3. A pi-chart which visualizes the average rarity of PAL for each element type. This visualization will use the element type and rarity features. 

### Part 1.4  (50 points)

Now, go ahead and try to scrape the features you need. 

Please show all the codes you have for web scrapping. Your current output data frame should include at least 4 features. (You do not need to finish all at this moment, although it is recommend to start earlier). **Please design your code in pipeline and clearly document each function.** See the Python Style Guide in Week 1 for proper documentation. It is also recommended to save the data you have scrapped. 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Pals Names, Rarety Number, List of Elements, List of work types
url = 'https://palworld.gg/pals'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')  

name_list = []
rarity_list = []
element_list = []
work_list = []


element_dict = {
     "elm1  element" : 'ice',
     "elm2  element" : 'dark',
     "elm3  element" : 'dragon',
     "elm4  element" : 'earth',
     "elm5  element" : 'leaf',
     "elm6  element" : 'electricity',
     "elm7  element" : 'water',
     "elm8  element" : 'fire',
     "elm9  element" : 'normal'
}

"Extract the necesary data from Pal's page"
for pal in soup.find_all('div', class_ = 'pal'):
    pal_elements = []
    pal_work_types = []
    pal_work_levels = []

    for name in pal.find_all('div', class_ = 'name'):
        if not name.find_parent('div', class_='rarity'):
            name = name.get_text()
            name_list.append(name)

    for rarity in pal.find_all('div', class_ = 'rarity'):
        for name in rarity.find_all('div', class_ = 'name'):
                rarity = name.get_text()
                rarity_list.append(rarity)

    for element in pal.find_all('div', class_ = 'element'):
        img_tag = element.find('img')
        element_key = img_tag['alt']
        pal_elements.append(element_dict[element_key]) 
    
    for work_type in pal.find_all('div', class_ = 'w'):
        img_tag = work_type.find('img')
        work_type = img_tag['alt']
        pal_work_types.append(work_type)


    for level in pal.find_all('div', class_ = 'level'): 
        all_levels = level.get_text()    
        pal_work_levels.append(all_levels)

    filtered_list = list(filter(lambda num: num != '0', pal_work_levels))
    
    merged_list = [] 
    for a, b in zip(pal_work_types, filtered_list): merged_list.append(f"{a} {b}")
    element_list.append(pal_elements)
    work_list.append(merged_list)



work_list.pop()
element_list.pop()


df = pd.DataFrame({
        'name': name_list,
        'rarity': rarity_list,
        'elements': element_list,
        'work types': work_list

    })

df.head(20)


,name,rarity,elements,work types
0,Anubis #100,Epic,[earth],"[Handiwork 4, Mining 3, Transporting 2]"
1,Arsox #42,Common,[fire],"[Deforesting 1, Kindling 2]"
2,Astegon #98,Epic,"[dragon, dark]","[Handiwork 1, Mining 4, Oil Extracting 3]"
3,Azurobe #82,Rare,"[water, dragon]",[Watering 3]
4,Beakon #73,Rare,[electricity],"[Gathering 1, Generating Electricity 2, Transp..."
5,Beegarde #50,Common,[leaf],"[Gathering 1, Deforesting 1, Handiwork 1, Farm..."
6,Bellanoir #112,Legendary,[dark],"[Handiwork 2, Medicine Production 4, Transport..."
7,Bellanoir Libero #112,Legendary,[dark],"[Handiwork 2, Medicine Production 4, Transport..."
8,Blazamut #96,Epic,[fire],"[Kindling 3, Mining 4]"
9,Blazamut Ryu #96,Epic,"[dragon, fire]","[Kindling 3, Mining 4]"
